In [156]:
import numpy as npq
import subprocess
import os
import re
import pandas as pd
import argparse

## Cython definitions

In [157]:
%load_ext Cython
#Don't merge this cell with the next. The next contains cython code, as it starts by %%cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [158]:
%%cython

# -*- coding: utf-8 -*-
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
# Copyright Guglielmo Saggiorato 2018
""" An "efficient" reader of LAMMPS dump files in cython.
Usage:
```import pyximport
pyximport.install()
from load_lammps import read_lammps
for time, column_names,data in read_lammps(filepath):
  pass
```
"""
__author__ = "Guglielmo Saggiorato" #Modified by Ivan Palaia
__copyright__ = "Copyright 2018, Guglielmo Saggiorato"
__credits__ = ["Guglielmo Saggiorato",]
__license__ = "GPLv3"
__version__ = "1.0"
__maintainer__ = "Guglielmo Saggiorato"
__email__ = "astyonax@gmail.com"
__status__ = "Production"

import numpy as np
cimport numpy as np
import pandas as pd
cimport cython
from libc.stdio cimport FILE, fopen, fwrite, fscanf, fclose, fprintf, fseek, ftell, SEEK_SET, rewind, fread

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def read_traj(str fname):
    """ Reads a LAMMPS dump file frame by frame yielding the data as a pandas DataFrame
This function automatically reads the fields/columns names and sets the pandas dataframe columns accordingly
For maximum performance, the bulk of the data is loaded without using any python object
In: fname [str]-- a file name as strings
Out: frame time [int], columns name [list], data [pandas.DataFrame], boxbounds [pandas.DataFrame]
Usage:
```import pyximport
pyximport.install()
from load_lammps import read_lammps
for time, column_names,data in read_lammps(filepath):
  pass
    """
    cdef str line,values
    cdef int i,N,T,j,cln,
    cdef list toarr,columns,
    cdef list columnsbox,
    cdef int Nbox,clnbox
    cdef double[:,:] data
    cdef double[:,:] box
    cdef double tmp
    cdef FILE * ptr_r

    fin = open(fname,'r')
    ptr_r = fopen(bytes(fname.encode('utf-8')), "r")
    line = fin.readline()
    while line:
        if 'ITEM: TIMESTEP' in line:
            # begin new timestep
            T = int(fin.readline())
            N = 0
            columns = []
        if 'ITEM: NUMBER OF ATOMS' in line:
            N = int(fin.readline())
        if 'ITEM: BOX BOUNDS' in line:
            Nbox=3
            columnsbox = ['lo','hi']
            clnbox = len(columnsbox)
            if not (Nbox and clnbox):
                raise StopIteration
            box = np.zeros((Nbox,clnbox),dtype='float64')#,dtype=[(j,'float64') for j in columns])

            fseek(ptr_r,int(fin.tell()),SEEK_SET)
            # loop over x, y and z coordinate
            for i in range(Nbox):
                for j in range(clnbox):
                    fscanf(ptr_r,"%le",&tmp)
                    box[i,j] = tmp#toarr[j]

            fin.seek(ftell(ptr_r))
            qbox  = pd.DataFrame(np.asarray(box),columns=columnsbox)
        if 'ITEM: ATOMS' in line:
            columns = line.split()[2:]
            cln = len(columns)
            if not (N and cln):
                raise StopIteration
            data = np.zeros((N,cln),dtype='float64')#,dtype=[(j,'float64') for j in columns])

            fseek(ptr_r,int(fin.tell()),SEEK_SET)
            # loop over particles
            for i in range(N):
                for j in range(cln):
                    fscanf(ptr_r,"%le",&tmp)
                    data[i,j] = tmp#toarr[j]

            fin.seek(ftell(ptr_r))
            q  = pd.DataFrame(np.asarray(data),columns=columns)
            yield T,columns,q,qbox

        line = fin.readline()

## Export clusters from pre-run trajectory as configuration file for new simulation

In [164]:
from ovito.io import *
from ovito.modifiers import *
from ovito.data import *
from ovito.pipeline import *
from ovito.vis import *
import PySide6.QtCore
import os.path

dens=0.20

'''
eT=0.0
TrajLogFilePath='/Users/ivan/Documents/SpinningClusters/Simulations/Test_5/Results_qA5_dp0.50_dens{:.3f}_eT{:.2f}/TrajLog_qA5_dp0.50_dens{:.2f}_eT{:.2f}_nA100000_rp0.15_ra0.15_ep-10.0_ea20.0_ce100_T1.0_100.xyz'.format(dens,eT,dens,eT)
timestep=300000
'''
eT=10.0
TrajLogFilePath='/Users/ivan/Documents/SpinningClusters/Simulations/Test_5/Results_qA5_dp0.50_dens{:.3f}_eT{:.2f}/TrajLog_qA5_dp0.50_dens{:.2f}_eT{:.2f}_nA100000_rp0.15_ra0.15_ep-10.0_ea20.0_ce100_T1.0_100.xyz'.format(dens,eT,dens,eT)
timestep=6000

# Define string managers
r1 = re.compile('[_]')
r2 = re.compile(r'(([-+]?\d+\.\d+)|([-+]?\d+))')

# Open ClustersList
filepattern = subprocess.check_output(" echo {:s} | sed 's/.*\/TrajLog_//' | sed 's/.xyz//'".format(TrajLogFilePath), shell=True)
filepattern = filepattern.decode("utf-8")[:-1]
Test5Path = subprocess.check_output(" echo {:s} | sed 's/Results_.*//' | sed 's/.xyz//'".format(TrajLogFilePath), shell=True)
Test5Path = Test5Path.decode("utf-8")[:-2]
ClustersListFilePath = '{:s}/Analysis/ClustersFiles/ClustersList_{:s}_ts{:d}.dat'.format(Test5Path, filepattern, timestep)
ClustersListDf = pd.read_csv(ClustersListFilePath, sep=' ')
AvgRg = ClustersListDf['Rg'].mean()

# Get nA and dens, compute box length
for s in r1.split(filepattern):
    if s.startswith('nA'):
        nA = int(r2.split(s)[1])
    if s.startswith('qA'):
        qA = int(r2.split(s)[1])
    if s.startswith('dens'):
        dens = float(r2.split(s)[1])
    if s[0].isdigit():
        real = int(s)
TwoL=np.sqrt(nA/dens)
Lxlo=-0.5*TwoL
Lxhi=0.5*TwoL
    
    
# Analyse cluster with Ovito
if True:
# Data import:
    pipeline = import_file(TrajLogFilePath, multiple_frames=True)

    # Select type:
    pipeline.modifiers.append(SelectTypeModifier(types={2, 3}))

    # Delete selected:
    pipeline.modifiers.append(DeleteSelectedModifier())

    # Cluster analysis:
    pipeline.modifiers.append(ClusterAnalysisModifier(
        cutoff=1.15,
        sort_by_size=True,
        unwrap_particles=True,  # needs to stay true, otherwise omega&AngMom per cluster will be wrong
        compute_com=True,
        compute_gyration=True,
        cluster_coloring=True))

    # Export Cluster analysis    
    for ThisFrame in range(pipeline.source.num_frames):
        data = pipeline.compute(frame=ThisFrame)
        if data.attributes['Timestep'] == timestep:
            print(timestep)
            # check that no particle went missing
            if data.particles.count != nA:
                print("Missing atoms (only {:d} present) in timestep {:d}, file\n {:s}".format(data.particles.count, data.attributes['Timestep'], filestring))            
                break
            ClustersOvito = data.tables['clusters']
            assert (ClustersOvito.x[:] == np.arange(1, len(ClustersOvito.x[:]) + 1, 1)).all(), "ClusterIDs are weird... Add additional check in the ClustersRotation computation (AngMom and Omega) "
            ClustersDataDf = pd.DataFrame(ClustersOvito.xy(), columns=['ClusterID', 'Size'])
            ClustersDataDf['Xcm'] = ClustersOvito['Center of Mass'][:, 0]
            ClustersDataDf['Ycm'] = ClustersOvito['Center of Mass'][:, 1]
            ClustersDataDf['Gxx'] = ClustersOvito['Gyration Tensor'][:, 0]  
            # the gyration tensor is computed as in lammps, for instance: Gxx =  1/M sum(mi xi^2), where xi is the position of particle i wrt the CM of its cluster
            ClustersDataDf['Gyy'] = ClustersOvito['Gyration Tensor'][:, 1]
            ClustersDataDf['Gxy'] = ClustersOvito['Gyration Tensor'][:, 3]
            ClustersDataDf['Rg'] = np.sqrt(ClustersDataDf['Gxx'].values + ClustersDataDf['Gyy'].values)
            
            # Delete clusters smaller than 10 colloids. Delete clusters that are too close to boundaries.    
            print(len(ClustersDataDf.index))
            ClustersDataDf = ClustersDataDf[ClustersDataDf['Size']>=10]
            margine=4
            ClustersDataDf = ClustersDataDf[(ClustersDataDf['Xcm']>Lxlo+margine*ClustersDataDf['Rg']) & (ClustersDataDf['Ycm']>Lxlo+margine*ClustersDataDf['Rg']) & (ClustersDataDf['Xcm']<Lxhi-margine*ClustersDataDf['Rg']) & (ClustersDataDf['Ycm']<Lxhi-margine*ClustersDataDf['Rg']) ]
            AcceptableClustersID = ClustersDataDf['ClusterID'].values
            print(len(ClustersDataDf.index))
            
            break

6000
2598
773


In [165]:
# define numpy arrays because ovito is super-slow in looping through particles
dataparticlesidentifiers=np.array(data.particles.identifiers[:])
dataparticlescluster=np.array(data.particles.cluster[:])

# list particles belonging to each cluster    
CentralParticlesID=[[] for i in range(len(AcceptableClustersID))]
for i in range(len(dataparticlesidentifiers)):
    if i%10000==0:
        print(i)
    for j in range(len(AcceptableClustersID)):
        if dataparticlescluster[i] == AcceptableClustersID[j]:
            CentralParticlesID[j].append(dataparticlesidentifiers[i])
#ParticlesID=[ np.array([[x, x+1, x+2, x+3, x+4, x+5] for x in Y]).flatten() for Y in CentralParticlesID]
ClustersDf=pd.DataFrame([[AcceptableClustersID[i], CentralParticlesID[i], ClustersDataDf['Xcm'].values[i],  ClustersDataDf['Ycm'].values[i]] for i in range(len(AcceptableClustersID))], columns=['ClusterID','CentralParticlesID','Xcm','Ycm'])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [166]:
# Miguel's solution (much faster, but sometimes different results, didn't investigate)

a,b,c=dataparticlesidentifiers,AcceptableClustersID,dataparticlescluster

sortingindices=np.argsort(dataparticlescluster)
cluster_id,cluster_size=np.unique(dataparticlescluster[sortingindices],return_counts=True)
cluster_ij=np.cumsum(cluster_size)
get_cluster=lambda i: np.sort(a[d[cluster_ij[i-1] if i>0 else 0:cluster_ij[i]]])

g=[get_cluster(i-1) for i in b]

#b[0],cluster_id[0],get_cluster(0)[:10],CentralParticlesID[0][:10]

for a,b in zip(CentralParticlesID,g):
    #print(np.array(a)-np.array(b))
    assert (np.array(a)==b).all()

AssertionError: 

In [168]:
folder= '/Users/ivan/Documents/SpinningClusters/Simulations/Test_6/Input/Configurations/ConfigCluFrom_{:s}_ts{:d}/'.format(filepattern,timestep)
try:
    os.makedirs(folder)
except:
    pass

for currenttime, columns, data, box in read_traj(TrajLogFilePath):
    if currenttime>timestep:
        break
    if currenttime==timestep:
        print(currenttime)
        print(len(ClustersDf.index))
        for cid in ClustersDf['ClusterID']:
            print(cid)
            df=ClustersDf[ClustersDf['ClusterID']==cid].iloc[0]
            CentralAtomsList = df.CentralParticlesID
            Xcm = df.Xcm
            Ycm = df.Ycm
            configfile='{:s}Config_C{:d}.dat'.format(folder,cid)
            with open(configfile, 'w') as f:
                f.write("""LAMMPS Description 
 
 	 {:d} atoms 
 	 0 bonds 
 	 0 angles 
 	 0 dihedrals 
 	 0 impropers 
 
 	 3 atom types 
 	 0 bond types 
 	 0 angle types 
 	 0 dihedral types 
 	 0 improper types 
 
 	 {:f} {:f} xlo xhi
 	 {:f} {:f} ylo yhi 
 	 -0.05 0.05 zlo zhi
 
Masses 
 
 	 1 0.60000 
 	 2 0.08000 
 	 3 0.08000 
 
Atoms

""".format( len(CentralAtomsList)*(qA+1), box['lo'].iloc[0], box['hi'].iloc[0], box['lo'].iloc[1], box['hi'].iloc[1])
                       )
                    
                inew=0
                mol=0
                for i in CentralAtomsList:
                    mol+=1
                    
                    # central atom
                    inew+=1
                    atomdata=data[np.isclose(data['id'],i, rtol=0, atol=0.1)].iloc[0]
                    typ=int(atomdata['type'])
                    assert typ==1, "ERROR: Type not 1"
                    xc = atomdata['x']-Xcm
                    yc = atomdata['y']-Ycm
                    vxc = 0  # atomdata['vx']
                    vyc = 0  # atomdata['vy']
                    f.write(" {:d} {:d} {:d} 0 {:f} {:f} 0 \n".format(inew, mol, typ, xc, yc)) # atom-ID molecule-ID atom-type q x y z
                    
                    # main patch
                    inew+=1
                    i+=1
                    atomdata=data[np.isclose(data['id'],i, rtol=0, atol=0.1)].iloc[0]
                    typ=int(atomdata['type'])
                    assert typ==2, "ERROR: Type not 2"
                    xp = atomdata['x']-Xcm
                    yp = atomdata['y']-Ycm
                    vxp = 0  # atomdata['vx']
                    vyp = 0   # atomdata['vy']
                    f.write(" {:d} {:d} {:d} 0 {:f} {:f} 0 \n".format(inew, mol, typ, xp, yp))
                    
                    # other patches
                    rprime = [xp-xc, yp-yc]
                    for j in [1,2,3,4]:
                        i+=1
                        inew+=1
                        typ=3
                        theta = 2*np.pi*j/qA
                        RotMatrix = np.array([[np.cos(theta),np.sin(theta)],[-np.sin(theta),np.cos(theta)]])
                        x,y = np.array([xc,yc]) + np.dot(RotMatrix,rprime)   # CM already removed
                        vx = 0
                        vy = 0
                        f.write(" {:d} {:d} {:d} 0 {:f} {:f} 0 \n".format(inew, mol, typ, x, y))


6000
773
2
4
11
12
16
18
19
21
23
25
28
33
34
38
39
40
41
43
44
45
47
48
49
52
53
54
59
60
61
63
64
68
69
70
72
73
74
76
81
82
83
84
85
86
87
88
89
91
92
94
95
96
97
99
101
103
104
105
108
109
110
111
112
114
117
119
121
122
123
124
126
127
130
133
135
136
140
142
143
144
145
146
147
148
149
150
152
154
156
158
159
160
162
163
166
167
168
169
171
172
173
174
175
177
179
180
181
185
186
187
189
190
191
193
196
197
199
200
201
203
205
206
207
208
209
210
211
213
214
215
216
219
220
221
225
226
227
229
230
231
233
238
240
241
242
243
244
248
249
250
251
252
255
256
258
259
261
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
281
283
284
285
288
289
290
291
292
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
311
312
316
317
318
319
320
321
322
324
327
328
329
330
331
332
334
336
337
338
339
340
342
343
344
345
346
347
349
350
351
353
354
355
357
358
359
361
363
365
366
367
368
369
370
371
372
375
376
377
378
379
380
381
382
383
385
386
388
389
390
391
392